In [1]:
import numpy as np
import tensorflow as tf
from glob import glob
from pinn import get_network
from pinn.utils import atomic_dress, get_atomic_dress, connect_dist_grad
from ase.collections import g2
from pinn.io import sparse_batch, load_qm9
from docs.notebooks.network_fns import preprocess_traintest_sets, train_and_evaluate_network, _generator, predict_energy
import matplotlib.pyplot as plt
from ase import Atoms
from pinn.optimizers import get
import time

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
physical_devices = tf.config.list_physical_devices()
tf.config.set_visible_devices(physical_devices[0], 'CPU')
tf.config.set_visible_devices([], 'GPU')

In [3]:
filelist = glob('/Users/miguelnavaharris/Project/QM9/*.xyz')
dataset = load_qm9(filelist, splits={'train':8, 'test':2})
dress, error = get_atomic_dress(dataset['train'], [1,6,7,8,9])

2022-07-20 23:22:33.789493: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-20 23:22:33.789598: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [4]:
def get_traintest_sets(batch_size):
    train_set = dataset['train'].shuffle(1000).apply(sparse_batch(batch_size))
    test_set = dataset['test'].apply(sparse_batch(batch_size))
    return (train_set, test_set)

In [5]:
params = {'optimizer': {'class_name': 'Adam', 'config': {'learning_rate': {'class_name': 'ExponentialDecay', 'config': {'initial_learning_rate': 0.0003, 'decay_steps': 10000, 'decay_rate': 0.994}}, 'clipnorm': 0.01}}, 'network': {'name': 'PiNet', 'params': {'depth': 4, 'rc': 4.0, 'atom_types': [1, 6, 7, 8, 9]}}, 'model': {'name': 'potential_model', 'params': {'learning_rate': 0.001, 'e_scale': 1, 'e_dress': dress}},'use_force':{}}

In [6]:
network = get_network(params['network'])

In [7]:
def preprocess_traintest_sets(train_set, test_set):
    for batch in train_set:
        batch = network.preprocess(batch)
        connect_dist_grad(batch)
    for batch in test_set:
        batch = network.preprocess(batch)
        connect_dist_grad(batch)

In [8]:
batch_size = 128
train_set, test_set = get_traintest_sets(batch_size)
preprocess_traintest_sets(train_set, test_set)

In [9]:
def train_and_evaluate_network(network=network, params=params, train_set=train_set, test_set=test_set, batch_size=256, epochs=1):


    # Instantiate an optimizer
    optimizer = get(params['optimizer'])
    # Define a loss function
    loss_fn = tf.keras.losses.mse
    # Define metrics
    train_loss_metric = tf.keras.metrics.MeanSquaredError()
    val_loss_metric = tf.keras.metrics.MeanSquaredError()
    train_err_metric = tf.keras.metrics.RootMeanSquaredError()
    val_err_metric = tf.keras.metrics.RootMeanSquaredError()
    

    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))
        start_time_epoch = time.time()
        hund_step_times = []
        val_errors = []


        # Iterate over the batches of the dataset.
        for step, batch in enumerate(train_set):

            # Open a GradientTape to record the operations run
            # during the forward pass, which enables auto-differentiation.
            with tf.GradientTape() as tape:

                # Run the forward pass of the layer.
                # The operations that the layer applies
                # to its inputs are going to be recorded
                # on the GradientTape.

                pred = network(batch, training=True)  # Logits for this minibatch

                ind = batch['ind_1']
                nbatch = tf.reduce_max(ind)+1
                pred = tf.math.unsorted_segment_sum(pred, ind[:, 0], nbatch)
                e_data = batch['e_data']

                if params['model']['params']['e_dress']:
                    e_data -= atomic_dress(batch, params['model']['params']['e_dress'], dtype=pred.dtype)
                    e_data *= params['model']['params']['e_scale']


                # Compute the loss value for this minibatch.
                loss_value = loss_fn(e_data, pred)

            # Use the gradient tape to automatically retrieve
            # the gradients of the trainable variables with respect to the loss.
            grads = tape.gradient(loss_value, network.trainable_weights)

            # Run one step of gradient descent by updating
            # the value of the variables to minimize the loss.
            optimizer.apply_gradients(zip(grads, network.trainable_weights))

            # Update the loss and error metrics
            train_loss_metric.update_state(e_data, pred)
            train_err_metric.update_state(e_data, pred)



            # Log every 100 batches.
            if step == 0:
                print(f"Initial loss (for one batch): {float(loss_value)}")
                print(f"Seen so far: {((step + 1) * batch_size)} molecules")



            elif step % 100 == 0:
                print(f"Training loss (for one batch) at step {step}: {float(loss_value)}")
                print(f"Seen so far: {((step + 1) * batch_size)} molecules")
                hund_step_times += [(time.time() - start_time_epoch)]
                print(f'Training time for 100 batches: {((hund_step_times[-1] - hund_step_times[-2]) if len(hund_step_times) > 1 else hund_step_times[-1])} s')



        print(f'Training time for epoch {epoch + 1}: {(time.time() - start_time_epoch)} s')



        

        # Run a validation loop at the end of each epoch
        print(f'Starting validation for epoch {(epoch + 1)}')

        for step, batch in enumerate(test_set):
            
            val_pred = network(batch, training=False)
            ind = batch['ind_1']
            nbatch = tf.reduce_max(ind)+1
            val_pred = tf.math.unsorted_segment_sum(val_pred, ind[:, 0], nbatch)
            e_data = batch['e_data']

            if params['model']['params']['e_dress']:
                e_data -= atomic_dress(batch, params['model']['params']['e_dress'], dtype=pred.dtype)
                e_data *= params['model']['params']['e_scale']


            # Update val metrics
            val_loss_metric.update_state(e_data, val_pred)
            val_err_metric.update_state(e_data, val_pred)


        print(f"Time taken for epoch {epoch + 1}: {(time.time() - start_time_epoch)} s")

        # Display metrics at the end of each epoch
        train_err = train_err_metric.result()
        print(f"Training err over epoch {(epoch + 1)}: {float(train_err)}")
        val_err = val_err_metric.result()
        val_errors.append(float(val_err))
        print(f"Validation err for epoch {(epoch + 1)}: {float(val_err)}")
        # Reset training metrics at the end of each epoch        
        train_err_metric.reset_states()
        val_err_metric.reset_states()

        if epoch > 3 and val_errors[-4] <= min(val_errors[-3:]):
                    return 'Stopped.'

        

In [10]:
train_and_evaluate_network(batch_size=batch_size, epochs=10000)


Start of epoch 0
Initial loss (for one batch): 9741.57421875
Seen so far: 128 molecules
Training loss (for one batch) at step 100: 1.584776759147644
Seen so far: 12928 molecules
Training time for 100 batches: 11.008339166641235 s
Training loss (for one batch) at step 200: 0.5289019346237183
Seen so far: 25728 molecules
Training time for 100 batches: 10.860275983810425 s
Training loss (for one batch) at step 300: 0.31758439540863037
Seen so far: 38528 molecules
Training time for 100 batches: 11.415031909942627 s
Training loss (for one batch) at step 400: 1.383690595626831
Seen so far: 51328 molecules
Training time for 100 batches: 11.730087041854858 s
Training loss (for one batch) at step 500: 0.25488150119781494
Seen so far: 64128 molecules
Training time for 100 batches: 11.979120969772339 s
Training loss (for one batch) at step 600: 0.3045615255832672
Seen so far: 76928 molecules
Training time for 100 batches: 11.373357057571411 s
Training loss (for one batch) at step 700: 0.21129250

IndexError: list index out of range

In [11]:
def _generator(molecule):
        data = {'coord': molecule.positions,
                'ind_1': np.zeros([len(molecule), 1]),
                'elems': molecule.numbers}
        yield data

def predict_energy(molecule):
        '''Takes an ASE Atoms object and outputs PiNet's energy prediction'''
        dtype=tf.float32
        dtypes = {'coord': dtype, 'elems': tf.int32, 'ind_1': tf.int32}
        shapes = {'coord': [None, 3], 'elems': [None], 'ind_1': [None, 1]}

        pred_dataset = tf.data.Dataset.from_generator(lambda:_generator(molecule), dtypes, shapes)

        for molecule in pred_dataset:
                molecule = network.preprocess(molecule)
                pred = network(molecule, training=False)
                ind = molecule['ind_1']
                nbatch = tf.reduce_max(ind)+1
                energy_prediction = tf.math.unsorted_segment_sum(pred, ind[:, 0], nbatch)
                energy_prediction_numpy = energy_prediction.numpy()[0]
        return energy_prediction_numpy

In [12]:
next(_generator(g2['CH4']))

{'coord': array([[ 0.      ,  0.      ,  0.      ],
        [ 0.629118,  0.629118,  0.629118],
        [-0.629118, -0.629118,  0.629118],
        [ 0.629118, -0.629118, -0.629118],
        [-0.629118,  0.629118, -0.629118]]),
 'ind_1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 'elems': array([6, 1, 1, 1, 1])}

In [13]:
predict_energy(g2['CH4'])

0.10940915

In [14]:
for i in train_set:
    print(i)
    a = i
    break

{'elems': <tf.Tensor: shape=(2310,), dtype=int32, numpy=array([6, 8, 6, ..., 1, 1, 1], dtype=int32)>, 'coord': <tf.Tensor: shape=(2310, 3), dtype=float32, numpy=
array([[-0.04232784,  1.399071  ,  0.02622217],
       [-0.0028356 , -0.02999014,  0.00863342],
       [-1.190231  , -0.64151204, -0.00798004],
       ...,
       [-0.3063433 , -2.4607291 ,  1.1791134 ],
       [ 0.01973908, -2.460475  , -1.2095277 ],
       [ 1.7551451 , -2.7061844 , -0.97355413]], dtype=float32)>, 'e_data': <tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-415.84818, -388.26443, -460.1791 , -356.4105 , -383.25824,
       -385.81726, -440.29184, -437.95004, -363.82178, -349.87012,
       -401.88684, -496.12506, -365.94238, -419.2043 , -398.15903,
       -383.1639 , -387.0078 , -436.714  , -458.93304, -387.07278,
       -348.97882, -419.14963, -399.84415, -385.89682, -437.952  ,
       -401.03882, -396.01077, -351.17123, -421.79355, -362.56934,
       -419.23734, -422.97028, -544.9652 , -383.71817, -389.

In [15]:
b = network.preprocess(a)
connect_dist_grad(b)

In [16]:
b_pred = network(b)
ind = b['ind_1']
nbatch = tf.reduce_max(ind)+1
b_pred = tf.math.unsorted_segment_sum(b_pred, ind[:, 0], nbatch)
e_data = b['e_data']
e_data -= atomic_dress(b, params['model']['params']['e_dress'], dtype=b_pred.dtype)
e_data *= params['model']['params']['e_scale']

In [17]:
b_pred

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.02334356,  0.13129595,  0.00388181,  0.09036437,  0.06984192,
        0.12849873,  0.03096673,  0.08801979,  0.05233441,  0.18315434,
        0.26323727,  0.14985794,  0.28263676,  0.13722047,  0.05402932,
        0.12795132,  0.12926355,  0.08792357,  0.15599254,  0.22791398,
        0.01809204,  0.05822349, -0.05284816,  0.07549483,  0.05241656,
       -0.09321058, -0.09120913,  0.18246794,  0.26677558,  0.08405244,
       -0.00292611,  0.08461102,  0.10344365, -0.051561  ,  0.13123792,
        0.34564012,  0.03771929,  0.12423572,  0.14190102,  0.11671156,
        0.16461325, -0.03116125,  0.12778872,  0.17329329,  0.10661355,
        0.292666  ,  0.20575708,  0.13029581,  0.13664913,  0.09482099,
        0.13021207,  0.19191644,  0.16570061,  0.01283294,  0.08395585,
       -0.08807528,  0.16191617, -0.0467796 ,  0.08784631,  0.02075377,
       -0.00814182,  0.10030571,  0.2241647 ,  0.00799146,  0.2084836 ,
        0.090573

In [18]:
e_data.numpy()

array([ 0.02023315,  0.00613403, -0.0206604 ,  0.01885986,  0.00390625,
        0.03814697, -0.00576782, -0.0791626 ,  0.03118896,  0.04138184,
        0.04013062, -0.02267456,  0.04064941,  0.0017395 , -0.03292847,
        0.09823608,  0.05517578, -0.0506897 ,  0.01779175, -0.00976562,
        0.01019287,  0.05648804, -0.04730225, -0.04141235, -0.08105469,
       -0.03439331, -0.01473999, -0.05215454,  0.00582886,  0.0760498 ,
       -0.03125   ,  0.03662109,  0.0402832 ,  0.00717163, -0.04452515,
        0.00723267, -0.06369019,  0.05422974, -0.02740479, -0.01364136,
        0.04977417,  0.02615356,  0.01586914,  0.00689697,  0.04431152,
        0.05307007, -0.01208496,  0.03683472, -0.02246094, -0.0625    ,
        0.04504395,  0.06854248,  0.07260132,  0.01083374, -0.04806519,
        0.01947021,  0.04568481, -0.00109863, -0.04293823, -0.05001831,
       -0.02484131, -0.00772095,  0.12619019, -0.06069946, -0.01196289,
       -0.07333374, -0.01507568, -0.00375366, -0.04779053, -0.01

In [19]:
import sklearn as sk
from sklearn.metrics import mean_squared_error
mean_squared_error(e_data, b_pred, squared=False)

0.13507892